# Importando as Bibliotecas

In [1]:
import ranking

import pandas as pd
import numpy as np
import string

from scipy import spatial

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

import time

# Constantes

In [2]:
COLUNAS = ("indice", "titulo_sa", "ean", "categoria")
LISTA_CATEGORIAS = ("celulares", "notebooks", "geladeiras", "fogoes", "tvs")

# Lendo os Datasets

In [3]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

# Funções

In [4]:
def criar_dicionario(indice, titulo_sa, ean, categoria):
    
    return {
            'indice' : indice, 'titulo_sa' : titulo_sa, 'ean' : ean, 'categoria' : categoria
           }


def criar_df_match(df_concat, ean_repetido):

    df_matches = pd.DataFrame(columns = COLUNAS)
    for ean in ean_repetido:

        # pega o indice da primeira linha com aquele EAN
        filtro = (df_concat['ean'] == ean)
        indice = next(iter(filtro.index[filtro]))

        dicionario = criar_dicionario(
                                    indice = indice,
                                    titulo_sa = df_concat.loc[indice]['titulo_sa'],
                                    ean = df_concat.loc[indice]['ean'],
                                    categoria = df_concat.loc[indice]['categoria']
                                    )

        df_matches = df_matches.append(dicionario, ignore_index = True)

    df_matches.sort_values('indice', inplace = True)
    df_matches.reset_index(drop = True, inplace = True)

    return df_matches


def salvar_resultado(nome, df):

    df.to_csv(f'Dados/Resultados/Cos_Rankeado/{nome}_métricas.csv', index = False)


def salvar_resultado_categoria(nome, df):

    for categoria in LISTA_CATEGORIAS:

        df[df["categoria"] == categoria].to_csv(f'Dados/Resultados/Cos_Rankeado/{categoria}/{nome}_métricas.csv', index = False)

# BoW

In [5]:
flag_cos = False
if flag_cos == True:

    for nome, df_teste in zip(arquivos, lista_df_teste):

        # remoção de pontuação e acentos
        ranking.fazer_pre_processamento(df_teste)

        # colocando os titulos em um dataframe com 1 coluna só
        df_concat = ranking.concatenar_df(df_teste)

        # calculando a quantidade máxima de palavras no título
        tam_max = ranking.calcular_tam_max(df_concat['titulo_sa'])

        # encontrando a lista de valores únicos de EAN repetidos
        vc = df_concat['ean'].value_counts()
        ean_repetido = vc[vc > 1].index.values

        # criando um dataframe apenas com 1 ocorrência de cada EAN repetido
        df_matches = criar_df_match(df_concat, ean_repetido)

        inicio_tempo = time.time()
        # calculando o BoW de cada título
        cv, titulo_bow = ranking.formatar_entrada_bow(df_concat['titulo_sa'], mf = tam_max)
        # calculando a distância entre os vetores (apenas linhas com match)
        resultado = ranking.calcular_dis_2_vetores_cond(titulo_bow, df_matches)
        final_tempo = time.time()
        tempo = final_tempo - inicio_tempo

        df_matches['tempo'] = tempo

        # colocando o resultado em ordem (menor distância até maior distância)
        indices, valores = ranking.ordenar_resultado(resultado)

        # calculando as métricas
        ranking.calcular_acuracia_k(df_matches, df_concat, indices)
        ranking.calcular_match_rank(df_matches, df_concat, indices)

        salvar_resultado(nome, df_matches)
        salvar_resultado_categoria(nome, df_matches)

c:\Users\llvs2\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
    a

NameError: name 'a' is not defined

# Resultados

In [6]:
def print_acu_k(df, flag = True):

    linha_1 = f"  | k-1\t | k-10\t| k-50\t|"

    if flag == True:
        linha_2 = f"1 | {df['k-1'].value_counts()[1]}\t | {df['k-10'].value_counts()[1]}\t| {df['k-50'].value_counts()[1]}\t|"
    else:
        linha_2 = f"1 | {df['k-1'].value_counts()[1]} | {df['k-10'].value_counts()[1]}\t| {df['k-50'].value_counts()[1]}\t|"
    linha_3 = f"0 | {df['k-1'].value_counts()[0]}\t | {df['k-10'].value_counts()[0]}\t| {df['k-50'].value_counts()[0]}\t|"

    print(f"{linha_1}\n{linha_2}\n{linha_3}")


def print_describe(df):

    print("")
    print(df[["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe())

## Geral

In [3]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_resultado = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/{arquivo}_métricas.csv")
    lista_df_resultado.append(df_treino)

### Aleátorio

#### 1:1

In [4]:
df_ale_1_1 = lista_df_resultado[0]
print_acu_k(df_ale_1_1)
print_describe(df_ale_1_1)

  | k-1	 | k-10	| k-50	|
1 | 376	 | 477	| 534	|
0 | 172	 | 71	| 14	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  548.000000  548.000000  548.000000  548.000000    548.000000
mean     0.686131    0.870438    0.974453    7.989051      0.750285
std      0.464488    0.336128    0.157925   27.656813      0.383151
min      0.000000    0.000000    0.000000    1.000000      0.002924
25%      0.000000    1.000000    1.000000    1.000000      0.500000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    2.000000      1.000000
max      1.000000    1.000000    1.000000  342.000000      1.000000


#### 5:1

In [100]:
df_ale_5_1 = lista_df_resultado[1]
print_acu_k(df_ale_5_1, False)
print_describe(df_ale_5_1)

  | k-1	 | k-10	| k-50	|
1 | 1506 | 1609	| 1636	|
0 | 152	 | 49	| 22	|

               k-1         k-10         k-50   match_rank  1/match_rank
count  1658.000000  1658.000000  1658.000000  1658.000000   1658.000000
mean      0.908323     0.970446     0.986731     3.205066      0.928224
std       0.288656     0.169404     0.114459    17.844705      0.231774
min       0.000000     0.000000     0.000000     1.000000      0.002294
25%       1.000000     1.000000     1.000000     1.000000      1.000000
50%       1.000000     1.000000     1.000000     1.000000      1.000000
75%       1.000000     1.000000     1.000000     1.000000      1.000000
max       1.000000     1.000000     1.000000   436.000000      1.000000


### Hard Negative

#### 1:1

In [115]:
df_hn_1_1 = lista_df_resultado[2]
print_acu_k(df_hn_1_1)
print_describe(df_hn_1_1)

  | k-1	 | k-10	| k-50	|
1 | 398	 | 511	| 559	|
0 | 186	 | 73	| 25	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  584.000000  584.000000  584.000000  584.000000    584.000000
mean     0.681507    0.875000    0.957192    9.827055      0.744075
std      0.466291    0.331002    0.202598   36.485142      0.387274
min      0.000000    0.000000    0.000000    1.000000      0.002331
25%      0.000000    1.000000    1.000000    1.000000      0.333333
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    3.000000      1.000000
max      1.000000    1.000000    1.000000  429.000000      1.000000


#### 5:1

In [102]:
df_hn_5_1 = lista_df_resultado[3]
print_acu_k(df_hn_5_1, False)
print_describe(df_hn_5_1)

  | k-1	 | k-10	| k-50	|
1 | 1088 | 1158	| 1177	|
0 | 93	 | 23	| 4	|

               k-1         k-10         k-50   match_rank  1/match_rank
count  1181.000000  1181.000000  1181.000000  1181.000000   1181.000000
mean      0.921253     0.980525     0.996613     2.138865      0.943770
std       0.269457     0.138246     0.058124    16.545500      0.198750
min       0.000000     0.000000     0.000000     1.000000      0.001912
25%       1.000000     1.000000     1.000000     1.000000      1.000000
50%       1.000000     1.000000     1.000000     1.000000      1.000000
75%       1.000000     1.000000     1.000000     1.000000      1.000000
max       1.000000     1.000000     1.000000   523.000000      1.000000


## Por Categoria

### Celulares

In [104]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_celulares = []
for arquivo in arquivos:

    df_celulares = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/celulares/{arquivo}_métricas.csv")
    lista_df_celulares.append(df_celulares)

In [105]:
df_ale_1_1_cel = lista_df_celulares[0]
df_ale_5_1_cel = lista_df_celulares[1]
df_hn_1_1_cel = lista_df_celulares[2]
df_hn_5_1_cel = lista_df_celulares[3]

In [112]:
print_acu_k(df_ale_1_1_cel)
print_describe(df_ale_1_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 44	 | 71	| 86	|
0 | 44	 | 17	| 2	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  88.000000  88.000000  88.000000   88.000000     88.000000
mean    0.500000   0.806818   0.977273   10.647727      0.591299
std     0.502865   0.397057   0.149887   37.477975      0.424425
min     0.000000   0.000000   0.000000    1.000000      0.002924
25%     0.000000   1.000000   1.000000    1.000000      0.160714
50%     0.500000   1.000000   1.000000    1.500000      0.750000
75%     1.000000   1.000000   1.000000    6.250000      1.000000
max     1.000000   1.000000   1.000000  342.000000      1.000000


In [116]:
print_acu_k(df_ale_5_1_cel)
print_describe(df_ale_5_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 181	 | 196	| 202	|
0 | 28	 | 13	| 7	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  209.000000  209.000000  209.000000  209.000000    209.000000
mean     0.866029    0.937799    0.966507    5.660287      0.887081
std      0.341439    0.242100    0.180352   22.659147      0.294071
min      0.000000    0.000000    0.000000    1.000000      0.004219
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000  237.000000      1.000000


In [117]:
print_acu_k(df_hn_1_1_cel)
print_describe(df_hn_1_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 63	 | 74	| 86	|
0 | 27	 | 16	| 4	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  90.000000  90.000000  90.000000   90.000000     90.000000
mean    0.700000   0.822222   0.955556    9.977778      0.737733
std     0.460825   0.384467   0.207235   32.201604      0.407882
min     0.000000   0.000000   0.000000    1.000000      0.003676
25%     0.000000   1.000000   1.000000    1.000000      0.270833
50%     1.000000   1.000000   1.000000    1.000000      1.000000
75%     1.000000   1.000000   1.000000    3.750000      1.000000
max     1.000000   1.000000   1.000000  272.000000      1.000000


In [130]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_hn_5_1_cel['k-1'].value_counts()[1]} | {df_hn_5_1_cel['k-10'].value_counts()[1]}\t| {df_hn_5_1_cel['k-50'].value_counts()[1]}\t|\n0 | {df_hn_5_1_cel['k-1'].value_counts()[0]}\t | {df_hn_5_1_cel['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_hn_5_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 190 | 197	| 204	|
0 | 14	 | 7	|	|

              k-1        k-10   k-50  match_rank  1/match_rank
count  204.000000  204.000000  204.0  204.000000    204.000000
mean     0.931373    0.965686    1.0    1.696078      0.942823
std      0.253441    0.182482    0.0    3.434938      0.213312
min      0.000000    0.000000    1.0    1.000000      0.030303
25%      1.000000    1.000000    1.0    1.000000      1.000000
50%      1.000000    1.000000    1.0    1.000000      1.000000
75%      1.000000    1.000000    1.0    1.000000      1.000000
max      1.000000    1.000000    1.0   33.000000      1.000000


### Fogões

In [3]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_fogoes = []
for arquivo in arquivos:

    df_fogoes = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/fogoes/{arquivo}_métricas.csv")
    lista_df_fogoes.append(df_fogoes)

In [4]:
df_ale_1_1_fg = lista_df_fogoes[0]
df_ale_5_1_fg = lista_df_fogoes[1]
df_hn_1_1_fg = lista_df_fogoes[2]
df_hn_5_1_fg = lista_df_fogoes[3]

In [133]:
print_acu_k(df_ale_1_1_fg)
print_describe(df_ale_1_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 144	 | 177	| 191	|
0 | 54	 | 21	| 7	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  198.000000  198.000000  198.000000  198.000000    198.000000
mean     0.727273    0.893939    0.964646    8.858586      0.785775
std      0.446491    0.308696    0.185140   32.736453      0.363779
min      0.000000    0.000000    0.000000    1.000000      0.003817
25%      0.000000    1.000000    1.000000    1.000000      0.500000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    2.000000      1.000000
max      1.000000    1.000000    1.000000  262.000000      1.000000


In [134]:
print_acu_k(df_ale_5_1_fg)
print_describe(df_ale_5_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 576	 | 637	| 644	|
0 | 75	 | 14	| 7	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  651.000000  651.000000  651.000000  651.000000    651.000000
mean     0.884793    0.978495    0.989247    2.795699      0.917635
std      0.319518    0.145173    0.103215   15.643127      0.235550
min      0.000000    0.000000    0.000000    1.000000      0.003690
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000  271.000000      1.000000


In [135]:
print_acu_k(df_hn_1_1_fg)
print_describe(df_hn_1_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 123	 | 168	| 189	|
0 | 78	 | 33	| 12	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  201.000000  201.000000  201.000000  201.000000    201.000000
mean     0.611940    0.835821    0.940299   12.253731      0.698558
std      0.488525    0.371363    0.237525   42.179738      0.397780
min      0.000000    0.000000    0.000000    1.000000      0.002545
25%      0.000000    1.000000    1.000000    1.000000      0.333333
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    3.000000      1.000000
max      1.000000    1.000000    1.000000  393.000000      1.000000


In [7]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_hn_5_1_fg['k-1'].value_counts()[1]}  | {df_hn_5_1_fg['k-10'].value_counts()[1]}\t| {df_hn_5_1_fg['k-50'].value_counts()[1]}\t|\n0 | {df_hn_5_1_fg['k-1'].value_counts()[0]}\t | {df_hn_5_1_fg['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_hn_5_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 302  | 351	| 354	|
0 | 52	 | 3	|	|

              k-1        k-10   k-50  match_rank  1/match_rank
count  354.000000  354.000000  354.0  354.000000    354.000000
mean     0.853107    0.991525    1.0    1.502825      0.905336
std      0.354500    0.091796    0.0    2.501273      0.235634
min      0.000000    0.000000    1.0    1.000000      0.026316
25%      1.000000    1.000000    1.0    1.000000      1.000000
50%      1.000000    1.000000    1.0    1.000000      1.000000
75%      1.000000    1.000000    1.0    1.000000      1.000000
max      1.000000    1.000000    1.0   38.000000      1.000000


### Geladeiras

In [141]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_gel = []
for arquivo in arquivos:

    df_gel = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/geladeiras/{arquivo}_métricas.csv")
    lista_df_gel.append(df_gel)

In [142]:
df_ale_1_1_gel = lista_df_gel[0]
df_ale_5_1_gel = lista_df_gel[1]
df_hn_1_1_gel = lista_df_gel[2]
df_hn_5_1_gel = lista_df_gel[3]

In [143]:
print_acu_k(df_ale_1_1_gel)
print_describe(df_ale_1_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 56	 | 69	| 76	|
0 | 22	 | 9	| 2	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  78.000000  78.000000  78.000000   78.000000     78.000000
mean    0.717949   0.884615   0.974359    5.974359      0.783290
std     0.452911   0.321553   0.159085   14.525369      0.363354
min     0.000000   0.000000   0.000000    1.000000      0.011111
25%     0.000000   1.000000   1.000000    1.000000      0.500000
50%     1.000000   1.000000   1.000000    1.000000      1.000000
75%     1.000000   1.000000   1.000000    2.000000      1.000000
max     1.000000   1.000000   1.000000   90.000000      1.000000


In [144]:
print_acu_k(df_ale_5_1_gel)
print_describe(df_ale_5_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 219	 | 226	| 230	|
0 | 12	 | 5	| 1	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  231.000000  231.000000  231.000000  231.000000    231.000000
mean     0.948052    0.978355    0.995671    1.913420      0.955251
std      0.222404    0.145838    0.065795    6.237429      0.193236
min      0.000000    0.000000    0.000000    1.000000      0.012658
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000   79.000000      1.000000


In [145]:
print_acu_k(df_hn_1_1_gel)
print_describe(df_hn_1_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 47	 | 70	| 73	|
0 | 28	 | 5	| 2	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  75.000000  75.000000  75.000000   75.000000     75.000000
mean    0.626667   0.933333   0.973333    6.160000      0.716552
std     0.486947   0.251124   0.162192   16.808974      0.383760
min     0.000000   0.000000   0.000000    1.000000      0.010526
25%     0.000000   1.000000   1.000000    1.000000      0.333333
50%     1.000000   1.000000   1.000000    1.000000      1.000000
75%     1.000000   1.000000   1.000000    3.000000      1.000000
max     1.000000   1.000000   1.000000   95.000000      1.000000


In [147]:
print_acu_k(df_hn_5_1_gel)
print_describe(df_hn_5_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 159	 | 161	| 163	|
0 | 6	 | 4	| 2	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  165.000000  165.000000  165.000000  165.000000    165.000000
mean     0.963636    0.975758    0.987879    2.127273      0.969040
std      0.187763    0.154269    0.109760    8.105376      0.163410
min      0.000000    0.000000    0.000000    1.000000      0.012195
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000   82.000000      1.000000


### Notebooks

In [149]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_not = []
for arquivo in arquivos:

    df_not = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/notebooks/{arquivo}_métricas.csv")
    lista_df_not.append(df_not)

In [150]:
df_ale_1_1_not = lista_df_not[0]
df_ale_5_1_not = lista_df_not[1]
df_hn_1_1_not = lista_df_not[2]
df_hn_5_1_not = lista_df_not[3]

In [151]:
print_acu_k(df_ale_1_1_not)
print_describe(df_ale_1_1_not)

  | k-1	 | k-10	| k-50	|
1 | 75	 | 88	| 97	|
0 | 25	 | 12	| 3	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  100.000000  100.000000  100.000000  100.000000    100.000000
mean     0.750000    0.880000    0.970000    7.380000      0.803959
std      0.435194    0.326599    0.171447   24.043639      0.356821
min      0.000000    0.000000    0.000000    1.000000      0.005780
25%      0.750000    1.000000    1.000000    1.000000      0.875000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.250000      1.000000
max      1.000000    1.000000    1.000000  173.000000      1.000000


In [152]:
print_acu_k(df_ale_5_1_not)
print_describe(df_ale_5_1_not)

  | k-1	 | k-10	| k-50	|
1 | 274	 | 288	| 292	|
0 | 20	 | 6	| 2	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  294.000000  294.000000  294.000000  294.000000    294.000000
mean     0.931973    0.979592    0.993197    3.112245      0.945204
std      0.252222    0.141633    0.082338   25.838386      0.205394
min      0.000000    0.000000    0.000000    1.000000      0.002294
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000  436.000000      1.000000


In [153]:
print_acu_k(df_hn_1_1_not)
print_describe(df_hn_1_1_not)

  | k-1	 | k-10	| k-50	|
1 | 95	 | 116	| 121	|
0 | 30	 | 9	| 4	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  125.000000  125.000000  125.000000  125.000000    125.000000
mean     0.760000    0.928000    0.968000    7.240000      0.800660
std      0.428802    0.259528    0.176708   27.314004      0.363950
min      0.000000    0.000000    0.000000    1.000000      0.004902
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000  204.000000      1.000000


In [154]:
print_acu_k(df_hn_5_1_not)
print_describe(df_hn_5_1_not)

  | k-1	 | k-10	| k-50	|
1 | 257	 | 266	| 271	|
0 | 15	 | 6	| 1	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  272.000000  272.000000  272.000000  272.000000    272.000000
mean     0.944853    0.977941    0.996324    3.235294      0.960045
std      0.228688    0.147146    0.060634   31.694514      0.172249
min      0.000000    0.000000    0.000000    1.000000      0.001912
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000  523.000000      1.000000


### TVs

In [155]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_tv = []
for arquivo in arquivos:

    df_tv = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/tvs/{arquivo}_métricas.csv")
    lista_df_tv.append(df_tv)

In [156]:
df_ale_1_1_tv = lista_df_tv[0]
df_ale_5_1_tv = lista_df_tv[1]
df_hn_1_1_tv = lista_df_tv[2]
df_hn_5_1_tv = lista_df_tv[3]

In [167]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_ale_1_1_tv['k-1'].value_counts()[1]}   | {df_ale_1_1_tv['k-10'].value_counts()[1]}\t| {df_ale_1_1_tv['k-50'].value_counts()[1]}\t|\n0 | {df_ale_1_1_tv['k-1'].value_counts()[0]}\t | {df_ale_1_1_tv['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_ale_1_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 57   | 72	| 84	|
0 | 27	 | 12	|	|

             k-1       k-10  k-50  match_rank  1/match_rank
count  84.000000  84.000000  84.0    84.00000     84.000000
mean    0.678571   0.857143   1.0     5.75000      0.738643
std     0.469830   0.352029   0.0    11.00698      0.394712
min     0.000000   0.000000   1.0     1.00000      0.021739
25%     0.000000   1.000000   1.0     1.00000      0.333333
50%     1.000000   1.000000   1.0     1.00000      1.000000
75%     1.000000   1.000000   1.0     3.00000      1.000000
max     1.000000   1.000000   1.0    46.00000      1.000000


In [168]:
print_acu_k(df_ale_5_1_tv)
print_describe(df_ale_5_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 256	 | 262	| 268	|
0 | 17	 | 11	| 5	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  273.000000  273.000000  273.000000  273.000000    273.000000
mean     0.937729    0.959707    0.981685    3.494505      0.943818
std      0.242091    0.197007    0.134334   14.277739      0.220835
min      0.000000    0.000000    0.000000    1.000000      0.007576
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000  132.000000      1.000000


In [169]:
print_acu_k(df_hn_1_1_tv)
print_describe(df_hn_1_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 70	 | 83	| 90	|
0 | 23	 | 10	| 3	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  93.000000  93.000000  93.000000   93.000000     93.000000
mean    0.752688   0.892473   0.967742   10.870968      0.794729
std     0.433788   0.311461   0.177642   47.936150      0.369310
min     0.000000   0.000000   0.000000    1.000000      0.002331
25%     1.000000   1.000000   1.000000    1.000000      1.000000
50%     1.000000   1.000000   1.000000    1.000000      1.000000
75%     1.000000   1.000000   1.000000    1.000000      1.000000
max     1.000000   1.000000   1.000000  429.000000      1.000000


In [170]:
print_acu_k(df_hn_5_1_tv)
print_describe(df_hn_5_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 180	 | 183	| 185	|
0 | 6	 | 3	| 1	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  186.000000  186.000000  186.000000  186.000000    186.000000
mean     0.967742    0.983871    0.994624    2.241935      0.971741
std      0.177162    0.126312    0.073324   13.736539      0.156123
min      0.000000    0.000000    0.000000    1.000000      0.005348
25%      1.000000    1.000000    1.000000    1.000000      1.000000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    1.000000      1.000000
max      1.000000    1.000000    1.000000  187.000000      1.000000
